<a href="https://colab.research.google.com/github/GENAMAGDY/Computer-Vision/blob/main/Project_DetectMotionFromVedios_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2 as cv
import cv2 as cv2_imshow
import numpy as np
import glob
import os
import math
import tensorflow as tf 
import cv2
from keras.utils import to_categorical
import numpy as np
import os.path
from keras.preprocessing import image as Img
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
from tqdm import tqdm

In [ ]:
base_model = InceptionV3(weights='imagenet',include_top=True)
model = Model( inputs=base_model.input,outputs=base_model.get_layer('avg_pool').output)

In [ ]:
def rescale_list(input_list, size):
        if len(input_list) >= size:
           skip = len(input_list) // size
        else : 
           l = size - len(input_list)
           for i in range(l+1):
              H,W,D = input_list[0].shape
              s = np.zeros((H,W,D))
              input_list.append(s)
              skip = len(input_list) // size

        output = [input_list[i] for i in range(0, len(input_list), skip)]
        return output[:size]

X_train = []
Y_train = []

listOfFiles = os.listdir(path=r'/content/drive/My Drive/CompetationCV/Training/')
number = 0
for entry in listOfFiles:
    Files = os.listdir('/content/drive/My Drive/CompetationCV/Training/' + entry)
    for video in Files:
        print('Train:', number)
        number += 1
        cap = cv.VideoCapture('/content/drive/My Drive/CompetationCV/Training/' + entry + '/' + video)
        frames = []
        while (True):
            ret, frame = cap.read()
            if ret == False:
              break
            else:
              frames.append(frame)
        rescale_list(frames, 40)
        sequence = []
        for image in frames:
            img = cv2.resize(image, (299, 299))
            x = Img.img_to_array(img)
            x = np.expand_dims(x, axis=0) 
            #The expand_dims() function is used to expand the shape of an array. Insert a new axis that will appear at the axis position in the expanded array shape.
            x = preprocess_input(x)
            #The preprocess_input function is meant to adequate your image to the format the model requires.
            features = model.predict(x)
            sequence.append(features[0])   
                
        X_train.append(sequence)
        if entry == 'Jumping':
            Y_train.append(1)
        elif entry == 'Tennis':
            Y_train.append(3)
        elif entry == 'Walking':
            Y_train.append(4)
        elif entry == 'Basketball':
            Y_train.append(2)
        elif entry == 'Diving':
            Y_train.append(0)
        cap.release()

        

y_train_categorical = to_categorical(Y_train).astype(np.integer)

In [ ]:
  
X_test = []
X_name = []
listOfFiles = os.listdir(path=r'/content/drive/My Drive/CompetationCV/Testing_set/')
number = 0
for video in listOfFiles:
        X_name.append(video)
        print('Test:', number)
        number += 1
        cap = cv.VideoCapture('/content/drive/My Drive/CompetationCV/Testing_set/' + video)
        frames = []
        while (True):
            ret, frame = cap.read()
            if ret == False:
              break
            else:
              frames.append(frame)
        rescale_list(frames, 40)
        sequence = []
        for image in frames:
            img = cv2.resize(image, (299, 299))
            x = Img.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features = model.predict(x)
            sequence.append(features[0])   
        X_test.append(sequence)
        
        cap.release()

        




In [ ]:
XX_test = np.empty((126,40,2048))
c=0
for C in X_test:
    XX_test[c] =np.resize(np.array(C) ,(40 , 2048))
    c+=1
y_train_categorical = to_categorical(Y_train).astype(np.integer)

In [ ]:
import numpy as np
import os.path
from keras.preprocessing import image as Img
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
from tqdm import tqdm
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from collections import deque
import sys
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time
import os.path
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
import h5py
import numpy as np
from keras.models import Model,Sequential
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, BatchNormalization, Dropout
from keras import regularizers
from keras.datasets import cifar10
from keras.layers import (Activation, Conv3D, Dense, Dropout)
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

from keras.optimizers import Adam , SGD, RMSprop
from keras.utils import np_utils, generic_utils
import tensorflow as tf
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
import random

In [ ]:
X= X_train
XX = np.empty((474 , 40,2048))
c=0
for C in X:
    XX[c] = np.resize(np.array(C) ,(40 , 2048))
    c+=1
y_train_categorical = to_categorical(Y_train).astype(np.integer)

In [ ]:
from keras.layers import Embedding

In [ ]:
model = Sequential()
'''''

model.add(LSTM(2048, return_sequences=False,input_shape=XX.shape[1:],dropout=0.5))
model.add(Dense(512, activation='relu',dropout=0.5))
model.add(Dropout(0.5))

''' Dropout is a regularization technique for reducing overfitting in artificial 
neural networks by preventing complex co-adaptations on training data.
 It is an efficient way of performing model averaging with 
neural networks. The term dilution refers to the thinning of the weights.'''
model.add(Dense(5, activation='softmax'))
optimizer = Adam(lr=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
print(model.summary())

model.fit(XX, y_train_categorical, batch_size=16,validation_split=0.1, verbose=1,epochs=50)

In [ ]:
prediction = model.predict(XX_test)
print(prediction)

In [ ]:
Y_test=[]
for i in prediction :
  maxx = np.max(i)
  if i[0] == maxx:
    Y_test.append(0)
  if i[1] == maxx:
    Y_test.append(1)
  if i[2] == maxx:
    Y_test.append(2)
  if i[3] == maxx:
    Y_test.append(3)
  if i[4] == maxx:
    Y_test.append(4)
print(Y_test)
import pandas as pd
df = pd.DataFrame({'Video': X_name, 'Label': Y_test})

df.to_csv('/content/drive/My Drive/CompetationCV/submit.csv' , index=False)
